In [1]:
%pip install seaborn
%pip install bokeh

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: seaborn in c:\users\user\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages (0.13.2)




[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Using cached bokeh-3.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached narwhals-2.16.0-py3-none-any.whl.metadata (14 kB)
  Using cached xyzservices-2025.11.0-py3-none-any.whl.metadata (4.3 kB)
Using cached bokeh-3.8.2-py3-none-any.whl (7.2 MB)
Using cached narwhals-2.16.0-py3-none-any.whl (443 kB)
Using cached xyzservices-2025.11.0-py3-none-any.whl (93 kB)
